# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests



# Import API key
from api_keys import geoapify_key


In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,13.97,31,17,3.07,US,1713817741
1,1,hobyo,5.3505,48.5268,29.43,73,100,5.63,SO,1713817753
2,2,badger,64.8000,-147.5333,6.05,78,96,0.95,US,1713817761
3,3,kavaratti,10.5669,72.6420,29.90,75,84,4.13,IN,1713817759
4,4,east london,-33.0153,27.9116,19.15,81,94,3.73,ZA,1713817809


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = "City",
                                      alpha = 0.6,
                                      size = "Humidity",
                                      tiles = "OSM"
                                     )


# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
# A max temperature lower than 26 degrees but higher than 20
# Cloudiness less than 4
# Wind speed less than 4.5 m/s
weather_filtered_df = city_data_df.loc[(city_data_df["Max Temp"] < 26)
                                       & (city_data_df["Max Temp"] > 20) 
                                       & (city_data_df["Cloudiness"] < 4)
                                       & (city_data_df["Wind Speed"] < 4.5)
                                      ]


# Drop any rows with null values
weather_filtered_df.dropna()

# Display sample data
weather_filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
90,90,eenhana,-17.4667,16.3333,20.59,31,0,3.21,NaN,1713818010
163,163,al jawf,29.5000,38.7500,22.51,10,2,2.43,SA,1713818030
186,186,santa cruz cabralia,-16.2781,-39.0247,25.50,85,0,2.08,BR,1713818038
190,190,ilhabela,-23.7781,-45.3581,21.61,85,0,2.21,BR,1713818039
244,244,sao vicente,-23.9631,-46.3919,22.74,80,0,1.32,BR,1713817873
288,288,tolanaro,-25.0319,46.9987,20.74,83,2,1.51,MG,1713818068
406,406,chabahar,25.2919,60.6430,25.98,73,0,3.68,IR,1713818103
455,455,grambling,32.5276,-92.7140,20.95,34,0,1.87,US,1713818118
498,498,belmonte,-15.8631,-38.8828,25.85,84,0,2.44,BR,1713818132
508,508,satsuma,30.8532,-88.0561,22.19,24,0,3.57,US,1713818135


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_filtered_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
90,eenhana,NaN,-17.4667,16.3333,31,
163,al jawf,SA,29.5000,38.7500,10,
186,santa cruz cabralia,BR,-16.2781,-39.0247,85,
190,ilhabela,BR,-23.7781,-45.3581,85,
244,sao vicente,BR,-23.9631,-46.3919,80,
288,tolanaro,MG,-25.0319,46.9987,83,
406,chabahar,IR,25.2919,60.6430,73,
455,grambling,US,32.5276,-92.7140,34,
498,belmonte,BR,-15.8631,-38.8828,84,
508,satsuma,US,30.8532,-88.0561,24,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000 
params = {"categories": "accommodation.hotel",
          "apiKey":geoapify_key
         }


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
eenhana - nearest hotel: Monte Carlo Guesthouse
al jawf - nearest hotel: No hotel found
santa cruz cabralia - nearest hotel: Campo Bahia Resort
ilhabela - nearest hotel: Vila Kebaya
sao vicente - nearest hotel: Pousada Vitória
tolanaro - nearest hotel: Hôtel Mahavokey
chabahar - nearest hotel: هتل سپیده
grambling - nearest hotel: Hampton Inn Ruston
belmonte - nearest hotel: No hotel found
satsuma - nearest hotel: La Quinta Inn & Suites by Wyndham Mobile Satsuma / Saraland


,City,Country,Lat,Lng,Humidity,Hotel Name
90,eenhana,NaN,-17.4667,16.3333,31,Monte Carlo Guesthouse
163,al jawf,SA,29.5000,38.7500,10,No hotel found
186,santa cruz cabralia,BR,-16.2781,-39.0247,85,Campo Bahia Resort
190,ilhabela,BR,-23.7781,-45.3581,85,Vila Kebaya
244,sao vicente,BR,-23.9631,-46.3919,80,Pousada Vitória
288,tolanaro,MG,-25.0319,46.9987,83,Hôtel Mahavokey
406,chabahar,IR,25.2919,60.6430,73,هتل سپیده
455,grambling,US,32.5276,-92.7140,34,Hampton Inn Ruston
498,belmonte,BR,-15.8631,-38.8828,84,No hotel found
508,satsuma,US,30.8532,-88.0561,24,La Quinta Inn & Suites by Wyndham Mobile Satsu...


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )


# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)